In [10]:
!pip install ultralytics
from ultralytics import YOLO
import cv2

ModuleNotFoundError: No module named 'ultralytics'

In [2]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.4.0.dev20240326+cu121 (NVIDIA GeForce RTX 4060 Laptop GPU)


In [3]:
model_p = YOLO("models/Yolo8LP/weights/best.pt")

In [6]:
model_p.track('dataset/test.mp4',save = True, conf = 0.2)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1466) c:\Users\zhout\My Drive (walcezu@umich.edu)\EECS442\442_final\dataset\test.mp4: 384x640 14 players, 2 referees, 208.9ms
video 1/1 (frame 2/1466) c:\Users\zhout\My Drive (walcezu@umich.edu)\EECS442\442_final\dataset\test.mp4: 384x640 14 players, 2 referees, 17.6ms
video 1/1 (frame 3/1466) c:\Users\zhout\My Drive (walcezu@umich.edu)\EECS442\442_final\dataset\test.mp4: 384x640 15 players, 2 referees, 16.9ms
video 1/1 (frame 4/1466) 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'player', 1: 'referee', 2: 'ball'}
 obb: None
 orig_img: array([[[ 86,  72,  99],
         [ 85,  71,  98],
         [ 78,  64,  91],
         ...,
         [109,  98, 116],
         [149, 140, 158],
         [192, 183, 201]],
 
        [[ 99,  85, 112],
         [101,  87, 114],
         [ 99,  85, 112],
         ...,
         [108,  97, 115],
         [148, 139, 157],
         [191, 182, 200]],
 
        [[116,  98, 135],
         [127, 109, 146],
         [136, 119, 154],
         ...,
         [112, 101, 119],
         [152, 143, 161],
         [191, 182, 200]],
 
        ...,
 
        [[ 32, 119,  89],
         [ 32, 119,  89],
         [ 32, 119,  89],
         ...,
         [ 32, 111,  85],
         [ 32, 111,  85],
         [ 32, 111,  85]],
 
        [[ 32, 119,  89],
         [ 32, 119,  89],
         [ 32, 119,  89],
       

In [ ]:
#using model_p to extract the color of the jersreys
import pandas as pd
from PIL import Image
import skimage


frame = cv2.imread('frame_image.jpg')
results_players = model_p('frame_image.jpg', conf=0.6)
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)                

palette_list = []                                                                
palette_interval = (0,5)                           
annotated_frame = cv2.imread('frame_image.jpg')
num_colors = 2
colors = {
    "Chelsea":[(41,71,138), (220,98,88)],
    "Man City":[(144,200,255), (188,199,3)] #(Players kit color, GK kit color)
}
colors_list = colors["Chelsea"] + colors["Man City"]
color_list_lab = [skimage.color.rgb2lab([i/255 for i in c]) for c in colors_list]                           

for player, count in enumerate(list(results_players[0].boxes.cls.cpu().numpy())):
    if int(count) == 0:
        #get the box of the player
        box = results_players[0].boxes.xyxy.cpu().numpy()[player, :]                   
        obj_img = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])] 
        width, height = obj_img.shape[1], obj_img.shape[0]
        #shrink the box to incorporate less of thd grass
        x1 = np.max([(width // 2) - (width // 6), 1])
        x2 = (width // 2) + (width // 6)
        y1 = np.max([(height // 3) - (height // 6), 1])
        y2 = (height // 3) + (height // 6)
        center_filter = obj_img[y1:y2, x1:x2]
        pil_img = Image.fromarray(np.uint8(center_filter))  
        #reduce the image size to just that of the small box
        reduced = pil_img.convert("P", palette=Image.Palette.WEB)
        #find the colors of the image
        palette = reduced.getpalette()                     
        palette = [palette[3 * n:3 * n + 3] for n in range(256)]      
        color_count = [(n, palette[m]) for n, m in reduced.getcolors()]                
        RGB_df = pd.DataFrame(color_count, columns = ['cnt', 'RGB']).sort_values(
                            by = 'cnt', ascending = False).iloc[
                            palette_interval[0]:palette_interval[1], :]
        palette = list(RGB_df.RGB) 
        #shows the small boxes on each player                                            
        annotated_frame = cv2.rectangle(annotated_frame,                
                                        (int(box[0]) + x1,
                                            int(box[1]) + y1),
                                        (int(box[0]) + x2,
                                            int(box[1]) + y2), (0,0,0), 2)
    
        cv2.imwrite('small_boxes.jpg', annotated_frame)  
        palette_list.append(palette)
        players_distance_features = []
        #iterate through each palette in the lists of palettes
        for palette in palette_list:
            palette_distance = []
            palette_lab = [skimage.color.rgb2lab([i/255 for i in color]) for color in palette]
            #iterate through each of the folors in thre paleette list
            for color in palette_lab:
                distance_list = []
                #iterate thtouhg each team color to find the color with the least distacne
                for c in color_list_lab:
                    distance = skimage.color.deltaE_cie76(color, c)                             
                    distance_list.append(distance)                                              
                palette_distance.append(distance_list)                                          
            players_distance_features.append(palette_distance)                
        players_teams_list = []

        for dist in players_distance_features:
            vote = []
            for dist_list in dist:
                team_idx = dist_list.index(min(dist_list)) // num_colors                   
                vote.append(team_idx)                                              
            players_teams_list.append(max(vote, key=vote.count))
print(players_teams_list)

annotated_frame = cv2.imread('frame_image.jpg')
for i, team_index in enumerate(players_teams_list):
    box = results_players[0].boxes.xyxy.cpu().numpy()[i,:]
    annotated_frame = cv2.rectangle(annotated_frame, (int(box[0]), int(box[1])), 
                                    (int(box[2]), int(box[3])), (0, 255, 0), 2) 
    x_center = int((box[0] + box[2]) / 2)
    y_top = int(box[1])
    position = (x_center - 50, y_top - 10)
    
    team_color = (255, 255, 255) 
    if team_index == 0:
        team_color = (0, 0, 255) 
    elif team_index == 1:
        team_color = (0, 255, 0) 
    
    annotated_frame = cv2.putText(annotated_frame, f"Team: {players_teams_list[team_index]}", position,
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, team_color, 2, cv2.LINE_AA)
cv2.imwrite('players_with_predictions.jpg', annotated_frame)

In [4]:
model_f = YOLO("models/Yolo8MF/weights/best.pt")
# model_f.predict('dataset/test.mp4',save = True,conf = 0.2)

In [5]:
result = model_f.predict('frame_image.jpg',save = True,conf = 0.2)


image 1/1 c:\Users\zhout\My Drive (walcezu@umich.edu)\EECS442\442_final\frame_image.jpg: 384x640 1 TRC, 1 TR6MC, 1 TL6MC, 1 TR6ML, 1 TL6ML, 1 TR18MC, 1 TR18ML, 1 TRArc, 1 TLArc, 1 RMC, 1 BRArc, 89.9ms
Speed: 0.0ms preprocess, 89.9ms inference, 733.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict6


In [6]:

for r in result:
    print(r.boxes.xyxy)
    print(r.boxes.cls)
    box = r.boxes.xyxy.cpu().numpy()
    cls = r.boxes.cls.cpu().numpy()
print(model_f.names)


tensor([[  42.4115,   73.6438,   72.8913,  104.8323],
        [  30.9616,   98.6347,   60.1895,  128.7405],
        [1035.0713,   82.4019, 1065.2412,  112.1971],
        [ 843.8967,  431.2320,  874.1234,  461.2748],
        [  83.1459,  155.1336,  111.9179,  185.4937],
        [1113.4604,  155.9391, 1143.8699,  187.1442],
        [1487.4834,   75.8499, 1517.5239,  106.3433],
        [ 648.0400,  159.6784,  677.6522,  190.0375],
        [ 687.8273,  105.7599,  718.3786,  135.6172],
        [ 992.0807, 1043.7299, 1022.0110, 1073.2054],
        [ 675.6902,   82.1380,  706.1970,  111.9479]], device='cuda:0')
tensor([ 5.,  3.,  8., 13., 11.,  6.,  1., 10.,  2., 26.,  4.], device='cuda:0')
{0: 'TLC', 1: 'TRC', 2: 'TR6MC', 3: 'TL6MC', 4: 'TR6ML', 5: 'TL6ML', 6: 'TR18MC', 7: 'TL18MC', 8: 'TR18ML', 9: 'TL18ML', 10: 'TRArc', 11: 'TLArc', 12: 'RML', 13: 'RMC', 14: 'LMC', 15: 'LML', 16: 'BLC', 17: 'BRC', 18: 'BR6MC', 19: 'BL6MC', 20: 'BR6ML', 21: 'BL6ML', 22: 'BR18MC', 23: 'BL18MC', 24: 'BR18ML', 

TLC: top left
TRC: top right,

tr6mc: Top right 6 corner
tl6mc: top left 6 corner
tr6ml: top right 6 base
tl6ml: top left 6 base

tr18mc: Top right 18 corner
tl18mc: top left 18 corner
tr18ml: top right 18 base
tl18ml: top left 18 base

TRarc: top right arc 
TLarc: top left arc

RML: midfield right corner
RMC: midfield right circl corner

LMC: midfiled left circle corner
LML: midfield left corner

BLC: bottom left
BRC: bottom right

br6mc: bottom right 6 corner
bl6mc: bottom left 6 corner
br6ml: bottom right 6 base
bl6ml: bottom left 6 base

br18mc: bottom right 18 corner
bl18mc: bottom left 18 corner
br18ml: bottom right 18 base
bl18ml: bottom left 18 base

BRarc: bottom right arc 
BLarc: bottom left arc

In [13]:
import cv2

# Variable to store the coordinates
point = None
points = []
# Mouse callback function
def get_coordinates(event, x, y, flags, param):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)

# Load the image
image = cv2.imread('field_graph.jpg')

# Create a window and bind the mouse callback function
cv2.namedWindow('image')
cv2.setMouseCallback('image', get_coordinates)

# Main loop
while True:
    # Display the image
    cv2.imshow('image', image)

    # Wait for a key press
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed, break from the loop
    if key == ord('q'):
        break

    # If a point is selected, print its coordinates
    if point is not None:
        points.append(point)
        point = None

# Clean up
cv2.destroyAllWindows()

In [14]:
points

[(28, 485),
 (28, 25),
 (76, 198),
 (75, 320),
 (27, 197),
 (28, 320),
 (148, 138),
 (147, 382),
 (27, 138),
 (28, 382),
 (149, 202),
 (148, 316),
 (352, 25),
 (351, 193),
 (351, 323),
 (351, 484),
 (676, 486),
 (674, 25),
 (628, 198),
 (627, 320),
 (675, 199),
 (675, 320),
 (555, 138),
 (556, 382),
 (675, 138),
 (676, 381),
 (553, 205),
 (554, 315)]

In [15]:
def draw_dot_with_number(image, coordinate, index):
    # Draw a dot
    cv2.circle(image, coordinate, 5, (0, 255, 0), -1)  # Green dot

    # Draw the index number
    cv2.putText(image, str(index), (coordinate[0] - 10, coordinate[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)  # Red number

# Load the image
image = cv2.imread('field_graph.jpg')

# List of coordinates
  # Example coordinates

# Plot dots with numbers for each coordinate
for i, coord in enumerate(points):
    draw_dot_with_number(image, coord, i)

# Display the image
cv2.imshow('Dots with Numbers', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
import numpy as np
np.save("field_coord.npy", np.array(points))

In [17]:
transform_coord = np.load('field_coord.npy')

In [31]:
transform_coord

array([[ 28, 320],
       [ 75, 320],
       [ 27, 138],
       [351, 193],
       [148, 316],
       [148, 138],
       [ 28,  25],
       [149, 202],
       [ 76, 198],
       [553, 205],
       [ 27, 197]])

In [18]:
cls = cls.astype(np.int32)
transform_coord = transform_coord[cls]


In [19]:
box
pts = np.mean(box.reshape(-1, 2, 2), axis=1)

In [20]:
pts

array([[     57.651,      89.238],
       [     45.576,      113.69],
       [     1050.2,      97.299],
       [     859.01,      446.25],
       [     97.532,      170.31],
       [     1128.7,      171.54],
       [     1502.5,      91.097],
       [     662.85,      174.86],
       [      703.1,      120.69],
       [       1007,      1058.5],
       [     690.94,      97.043]], dtype=float32)

In [21]:
pts = pts.astype(np.int16)

In [22]:
homography, _ = cv2.findHomography(pts, transform_coord)

In [29]:
image = cv2.imread('field_graph.jpg')
height, width = image.shape[:2]
image = cv2.imread('frame_image.jpg')

warped_image = cv2.warpPerspective(image, homography, (width, height))


In [30]:
cv2.imwrite('processed_image.jpg', warped_image)

True